Hardware Preview

In [1]:
!lscpu
# Expect Intel Xeon Chip @2.0Ghz with 2 cores 4 threads, 2xT4 wont give you this much CPU power, but more GPU power in predicting samples.
!nvidia-smi 
# nVidia P100 expected, 2xT4 requires setup encode_multi_process, read it here https://www.sbert.net/docs/package_reference/SentenceTransformer.html
!python -V 
# Python 3.7.12
!cmake --version
# cmake version 3.22.5
!nvcc --version

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          2
On-line CPU(s) list:             0,1
Thread(s) per core:              2
Core(s) per socket:              1
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           85
Model name:                      Intel(R) Xeon(R) CPU @ 2.00GHz
Stepping:                        3
CPU MHz:                         2000.184
BogoMIPS:                        4000.36
Hypervisor vendor:               KVM
Virtualization type:             full
L1d cache:                       32 KiB
L1i cache:                       32 KiB
L2 cache:                        1 MiB
L3 cache:                        38.5 MiB
NUMA node0 CPU(s):               0,1
Vulnerability 

Install needed library

In [ ]:
!pip install -q visen
!pip install -q sentence-transformers
!pip install -q "modin[all]"
!pip install onnx
!pip install onnxruntime

In [ ]:
# !pip install -q pyvi
# !pip install -q pretty-confusion-matrix
# !pip install -q Cython
# !pip uninstall -q -y lightgbm
# !apt-get -q install -y libboost-all-dev
# !git clone --quiet --recursive https://github.com/Microsoft/LightGBM
# !git clone --quiet https://github.com/coccoc/coccoc-tokenizer.git

# !cd coccoc-tokenizer/ && mkdir build
# !cd coccoc-tokenizer/ && cd build && cmake -DBUILD_PYTHON=1 ..
# !cd coccoc-tokenizer/ && cd build && make install
# !cd coccoc-tokenizer/python && python setup.py install

# !cd LightGBM/ && rm -r build
# !cd LightGBM/ && mkdir build
# !cd LightGBM/ && cd build && cmake -DUSE_GPU=1 -DOpenCL_LIBRARY=/usr/local/cuda/lib64/libOpenCL.so -DOpenCL_INCLUDE_DIR=/usr/local/cuda/include/ ..
# !cd LightGBM/ && cd build && make -j$(nproc)
# !cd LightGBM/python-package/;python setup.py install --precompile
# !mkdir -p /etc/OpenCL/vendors && echo "libnvidia-opencl.so.1" > /etc/OpenCL/vendors/nvidia.icd
# !rm -r LightGBM

In [ ]:
import re
from unicodedata import normalize
from string import punctuation
import visen
import pickle

import gc
import warnings
warnings.filterwarnings("ignore")
import scipy
import sys
import os
from time import time

os.environ["MODIN_ENGINE"] = "dask"
os.environ["MODIN_CPUS"] = "16"
# import pandas as pd
import modin.pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, log_loss
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import LabelEncoder

import torch
from torch import optim

# from CocCocTokenizer import PyTokenizer
# tokenizer = PyTokenizer(load_nontone_data=True).word_tokenize
# import lightgbm as lgb
# from pretty_confusion_matrix import pp_matrix


In [ ]:
opening_ls = ['[', '{', '⁅', '〈', '⎡', '⎢', '⎣', '⎧', '⎨', '⎩', '❬', '❰', '❲', '❴', '⟦', '⟨', '⟪', '⟬', '⦃', '⦇', '⦉',
              '⦋', '⦍', '⦏', '⦑', '⦓', '⦕', '⦗', '⧼', '⸂', '⸄', '⸉', '⸌', '⸜', '⸢', '⸤', '⸦', '〈', '《', '「', '『',
              '【', '〔', '〖', '〘', '〚', '﹛', '﹝', '［', '｛', '｢', '｣']

closing_ls = [']', '}', '⁆', '〉', '⎤', '⎥', '⎦', '⎫', '⎬', '⎭', '❭', '❱', '❳', '❵', '⟧', '⟩', '⟫', '⟭', '⦄', '⦈', '⦊',
              '⦌', '⦎', '⦐', '⦒', '⦔', '⦖', '⦘', '⧽', '⸃', '⸅', '⸊', '⸍', '⸝', '⸣', '⸥', '⸧', '〉', '》', '」', '』',
              '】', '〕', '〗', '〙', '〛', '﹜', '﹞', '］', '｝', '｣']

opening_bracket = {key: '(' for key in opening_ls}
closing_bracket = {key: ')' for key in closing_ls}

opening_bracket_pattern = {f"\\{key}": "(" for key in opening_ls}
closing_bracket_pattern = {f"\\{key}": ")" for key in closing_ls}

### constant
PUNC = '!\"#$&()*+,-–−./:;=?@[\]^_`{|}~”“`°²ˈ‐ㄧ‛∼’'  # remove <> for number_sym and unknown_sym
re_num_and_decimal = '[0-9]*[,.\-]*[0-9]*[,.\-]*[0-9]*[.,\-]*[0-9]*[,.\-]*[0-9]+[.,]?'
re_unknown = '[a-z]+[\d]+[\w]*|[\d]+[a-z]+[\w]*'
re_vnese_txt = r'[^a-z0-9A-ZàáãạảăắằẳẵặâấầẩẫậèéẹẻẽêềếểễệđìíĩỉịòóõọỏôốồổỗộơớờởỡợùúũụủưứừửữựỳỵỷỹýÀÁÃẠẢĂẮẰẲẴẶÂẤẦẨẪẬÈÉẸẺẼÊỀẾỂỄỆĐÌÍĨỈỊÒÓÕỌỎÔỐỒỔỖỘƠỚỜỞỠỢÙÚŨỤỦƯỨỪỬỮỰỲỴỶỸÝ\s|_]'
special_punc = {'”': '"', '': '', "’": "'", "`": "'"}


def sizeof_fmt(num, suffix='B'):
    '''
    Utility function to quickly view memory usage
    :arg num: number of bytes
    :arg suffix: suffix of the number
    :return: string of the number with suffix

    by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified
    '''
    for unit in ['', 'Ki', 'Mi', 'Gi', 'Ti', 'Pi', 'Ei', 'Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

def replace_all(replacer: dict, txt: str) -> str:
    """
    Replace all the keys in the dictionary with their respective values.
    :param replacer: dictionary of keys are the words to be replaced and values are the words to replace them with
    :param txt: subject string
    :return: string after replacement
    """
    for old, new in replacer.items():
        txt = txt.replace(old, new)
    return txt


def replace_num(txt: str) -> str:
    """
    Replace all the numbers in the text with blank
    :param text: subject string
    :return: string after replacement
    """
    text = re.sub(re_num_and_decimal, '', txt)
    return text


def replace_unknown(text: str) -> str:
    """
    Replace all the predefined unknown symbols in the text with blank
    :param text: subject string
    :return: string after replacement
    """
    text = re.sub(re_unknown, '', text)
    return text


def unicode_normalizer(text, forms: list = ['NFKC', 'NKFD', 'NFC', 'NFD']) -> str:
    """
    Normalize unicode text
    :param text: subject string
    :param forms: unicode normalization forms
    :return: string after normalization
    """
    for form in forms:
        text = normalize(form, text)
    return text


def normalize_bracket(text: str) -> str:
    """
    Normalize brackets in the text with predefined string for later use
    :param text: subject string
    :return: transformed string
    """
    text = replace_all(opening_bracket, text)
    text = replace_all(closing_bracket, text)
    text = re.sub(r"[\(\[].*?[\)\]]", "", text)
    return text


def remove_punc(text: str) -> str:
    """
    Remove punctuations in the text
    :param text: subject string
    :return: string after removal
    """
    r = re.compile(r'[\s{}]+'.format(re.escape(PUNC)))
    text = r.split(text)
    return ' '.join(i for i in text if i)


def norm(text: str) -> str:
    """
    Normalize text by removing punctuations, numbers, unknown symbols, brackets, and normalize unicode
    :param text: subject string
    :return: normalized string
    """
    text = str(text)
    text = text.lower()
    text = text.split('\n')[0]
    text = unicode_normalizer(text, ["NFKC", "NFKD", "NFD", "NFC"])
    text = replace_all(special_punc, text)
    text = normalize_bracket(text)
    text = replace_unknown(text)
    text = replace_num(text)
    text = remove_punc(text)
    text = re.sub(re_vnese_txt, "", text)
    text = text.strip()
    return visen.clean_tone(text)

In [ ]:
sample_txts = [
    '〖𝔰𝔬𝔪𝔢 𝔣𝔞𝔫𝔠𝔶 𝔭𝔯𝔬𝔡𝔲𝔠𝔱 𝔫𝔞𝔪𝔢〗',
    '𝖘𝖔𝖒𝖊 𝖋𝖆𝖓𝖈𝖞 𝖕𝖗𝖔𝖉𝖚𝖈𝖙 𝖓𝖆𝖒𝖊',
    '❰some product code which will overfit this sample❱ 𝓼𝓸𝓶𝓮 𝓯𝓪𝓷𝓬𝔂 𝓹𝓻𝓸𝓭𝓾𝓬𝓽 𝓷𝓪𝓶𝓮',
    '𝓈𝑜𝓂𝑒 𝒻𝒶𝓃𝒸𝓎 𝓅𝓇𝑜𝒹𝓊𝒸𝓉 𝓃𝒶𝓂𝑒',
    '{𝕤𝕠𝕞𝕖 𝕗𝕒𝕟𝕔𝕪 𝕡𝕣𝕠𝕕𝕦𝕔𝕥 𝕟𝕒𝕞𝕖',
    '☯😝ｓｏｍｅ ｆａｎｃｙ ｐｒｏｄｕｃｔ ｎａｍｅ☯😝',
    '𝐬𝐨𝐦𝐞 𝐟𝐚𝐧𝐜𝐲 𝐩𝐫𝐨𝐝𝐮𝐜𝐭 𝐧𝐚𝐦𝐞',
    '𝘀𝗼𝗺𝗲 𝗳𝗮𝗻𝗰𝘆 𝗽𝗿𝗼𝗱𝘂𝗰𝘁 𝗻𝗮𝗺𝗲',
    '𝘴𝘰𝘮𝘦 𝘧𝘢𝘯𝘤𝘺 𝘱𝘳𝘰𝘥𝘶𝘤𝘵 𝘯𝘢𝘮𝘦',
    '▄▀▄▀▄▀ 𝙨𝙤𝙢𝙚 𝙛𝙖𝙣𝙘𝙮 𝙥𝙧𝙤𝙙𝙪𝙘𝙩 𝙣𝙖𝙢𝙚 ▄▀▄▀▄▀',
    '🌌  🎀 𝚜𝚘𝚖𝚎 𝚏𝚊𝚗𝚌𝚢 𝚙𝚛𝚘𝚍𝚞𝚌𝚝 𝚗𝚊𝚖𝚎🌌  🎀 ',
    '(っ◔◡◔)っ ♥ some fancy product name ♥',
]

for sample in sample_txts:
    print(f'sample \"{sample}\", after normalize: \"{norm(sample)}\"')

Still normalize function but for DataFrame

In [ ]:
def dataframe_normalize(df: pd.DataFrame=None) -> pd.DataFrame:
    """
    Normalize dataframe
    :param df: Dataframe to normalize
    :return: Dataframe normalized
    """    
    assert 'sample' in df.columns, f"DataFrame with column name 'sample' expected, got {df.columns} "
    begin = time()

    df['sample'] = df['sample'].str.lower()
    df['sample'] = df['sample'].str.replace('[a-z]+[\d]+[\w]*|[\d]+[a-z]+[\w]*', '', regex=True) # replace all product code with blank
    df['sample'] = df['sample'].str.replace('[0-9]*[,.\-]*[0-9]*[,.\-]*[0-9]*[.,\-]*[0-9]*[,.\-]*[0-9]+[.,]?', '', regex=True) # replace all number with blank
    df['sample'] = df['sample'].replace(opening_bracket_pattern, regex=True) # normalize opening brackets, listed above
    df['sample'] = df['sample'].replace(closing_bracket_pattern, regex=True) # normalize closing brackets, listed above
    df['sample'] = df['sample'].str.replace("[\(\[].*?[\)\]]", '', regex=True) # remove all content inside brackets, eg: (Siêu sale 12/12)
    df['sample'] = df['sample'].str.normalize('NFKC') # Normalize unicode with NFKC standard 
    df['sample'] = df['sample'].str.normalize('NFKD')
    df['sample'] = df['sample'].str.normalize('NFC')
    df['sample'] = df['sample'].str.normalize('NFD')
#   # Remove non Vietnamese text, seems like unessesary right now, might consider remove later to improve run time.
    df['sample'] = df['sample'].str.replace(r"[^a-z0-9A-ZàáãạảăắằẳẵặâấầẩẫậèéẹẻẽêềếểễệđìíĩỉịòóõọỏôốồổỗộơớờởỡợùúũụủưứừửữựỳỵỷỹýÀÁÃẠẢĂẮẰẲẴẶÂẤẦẨẪẬÈÉẸẺẼÊỀẾỂỄỆĐÌÍĨỈỊÒÓÕỌỎÔỐỒỔỖỘƠỚỜỞỠỢÙÚŨỤỦƯỨỪỬỮỰỲỴỶỸÝ\s|_]", '', regex=True)
    df['sample'] = df['sample'].str.replace('đ', 'd').str.strip() # Special case where unicode normalizer does not consider 'đ' a combination of base unicode characters
    print('Total enlapsed time:', time()- begin, 'seconds')
    return df

##### Data preparation

In [ ]:
data = pd.read_csv('/kaggle/input/data-dl-final/shopee_data.csv', encoding='utf8', sep='\t')

In [ ]:
data = data.sample(frac=1)
data

In [ ]:
data = dataframe_normalize(data)
data

In [ ]:
data = data[~(data['label'] == 'unknown')]
data.dropna(inplace=True)
data['label'].value_counts()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_val, y_train , y_val = train_test_split(data['sample'], data['label'],train_size=0.7, random_state=42, shuffle=True)

In [ ]:
X_train

In [ ]:
y_train.value_counts()

In [ ]:
le = LabelEncoder()
y_train = torch.tensor(le.fit_transform(y_train.values)).cuda()
y_val = torch.tensor(le.transform(y_val.values)).cuda()

In [ ]:
from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('keepitreal/vietnamese-sbert',device='cuda')

X_train = X_train.tolist()
X_train = embedder.encode(X_train, batch_size=2048)
X_train = torch.from_numpy(X_train).float()
X_train = X_train.cuda()


X_val = X_val.tolist()
X_val = embedder.encode(X_val, batch_size=2048)
X_val = torch.from_numpy(X_val).float()
X_val = X_val.cuda()


In [ ]:
print(le.inverse_transform([0]))
print(le.inverse_transform([1]))
print(le.inverse_transform([2]))
print(le.inverse_transform([3]))

# THIS MODEL IS **EXTREMELY** SIMPLE, TRY TO BUILD A **BIT** MORE COMPLEX MODEL!

In [ ]:
from torch import nn
model = nn.Sequential(nn.Linear(768, 64),
                      nn.ReLU(),
                      nn.Linear(64, 64),
                      nn.ReLU(),
                      nn.Dropout(0.1),
                      nn.Linear(64, data['label'].nunique()),
                      nn.LogSoftmax(dim=1))

# transfer model to gpu for training acceleration
model = model.to('cuda')

# Negative Log-Likelihood Loss
criterion = nn.NLLLoss()

# Forward pass, get our logits
logps = model(X_train)

# Calculate the loss with the logits and the labels
loss = criterion(logps, y_train)
loss.backward()

# Optimizers require the parameters to optimize and a learning rate
optimizer = optim.Adam(model.parameters(), lr=0.002)

next(model.parameters()).is_cuda

In [ ]:
# 1/0
# Prevent below blocks from running when excute run all

In [ ]:
for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),key= lambda x: -x[1])[:20]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

In [ ]:
%%time
train_losses = []
test_losses = []
test_accuracies = []

epochs = 200
for e in range(epochs):
    optimizer.zero_grad()
    output = model.forward(X_train)
    loss = criterion(output, y_train)
    loss.backward()
    train_loss = loss.item()
    train_losses.append(train_loss)
    
    # Sample evaluation step:
    with torch.no_grad():
        log_ps = model(X_val)
        ps = torch.exp(log_ps)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == y_val.view(*top_class.shape)
        val_accuracy = torch.mean(equals.float())
    
    optimizer.step()
    model.train()
    
    if e % 5 == 0:
        print(f"Epoch: {e+1}/{epochs}.. ",
              f"Training Loss: {train_loss:.3f}.. ",
              f"Training Acc: {val_accuracy:.3f}.. ")


In [ ]:
new_sample = ['iphone 13 promax', 'phấn phủ hồng đất', 'áo khoác dạ']

def predict(model, batch, output = 'df', mask_inference=True):
    assert output in ['df', 'dict', 'list']
    batch_sample = [norm(i) for i in batch]
    if mask_inference:
        print(batch_sample)
    
    batch_sample = embedder.encode(batch_sample, batch_size = min(len(batch), 2048))
    batch_sample = torch.from_numpy(batch_sample).float().cuda()
    model.eval()
    with torch.no_grad():
        out_data = model(batch_sample)
        ps = torch.exp(out_data)
        pred = ps.max(1).indices.cpu().numpy()
        res = [le.inverse_transform([i])[0] for i in pred]
    
    if output == 'list':
        return res
    
    data_tuple = zip(batch, res)
    
    if output == 'df':
        return pd.DataFrame(data_tuple, columns=['name', 'prediction'])
    
    if output == 'dict':
        return dict(data_tuple)


predict(model, new_sample, output='dict', mask_inference=False)

In [ ]:
df = pd.read_csv('/kaggle/input/data-dl-final/test_DTDM.csv', sep=',', encoding='utf8', on_bad_lines='skip')
test_sample = df.name.to_list()

In [ ]:
%%timeit

res = predict(model, test_sample, mask_inference=False)

In [ ]:
res = predict(model, test_sample, mask_inference=False)

In [ ]:
res

In [ ]:
res['prediction'].value_counts()

In [ ]:
wrong_sample = res[res['prediction'] != 'Điện tử - Điện máy']['name'].to_list()
wrong_sample

if wrong_sample:
    print(predict(model, wrong_sample, output = 'dict'))
wrong_sample

In [ ]:
wrong_sample

In [ ]:
random = ['áo choàng đông']
predict(model,random , output = 'dict')

In [ ]:
model.state_dict()

In [ ]:
torch.save(model, 'classifier.pt')

In [ ]:
import pickle

In [ ]:
with open('label_encoder.pkl', 'wb') as encoder:
    pickle.dump(le, encoder)